In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
flight_aviable = {"london":"yes","paris":"no","tokyo":"yes","berlin":"yes"}

def get_flight(booking):
    print(f"Tool get_flight called for {booking}")
    if not booking:  # Kollar om booking är None eller tom sträng
        return "No booking info provided"
    airplain = booking.lower()
    return flight_aviable.get(airplain, "Unknown")

In [ ]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
flight_function = {
    "name": "get_flight",
    "description": "Check if the flight booking is available. Call this whenever you need to know if a customer can book a flight, for example when a customer asks 'Can we book now?'",
    "parameters": {
        "type": "object",
        "properties": {
            "booking": {
                "type": "string",
                "description": "The flight booking identifier or destination",
            },
        },
        "required": ["booking"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": price_function},{"type": "function", "function": flight_function}]

In [ ]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    
    flight = arguments.get('booking')
    if not flight:   # Hantera None eller tom sträng
        flight = ""
    
    aviable = get_flight(flight)
    
    response = {
        "role": "tool",
        "content": json.dumps({
            "destination_city": city,
            "price": price,
            "booking": flight,
            "aviable": aviable
        }),
        "tool_call_id": tool_call.id
    }
    
    return response  # Returnera endast response, inte city och flight


In [ ]:

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        
        # Anropa verktyget och få bara response (ingen tuple)
        tool_response = handle_tool_call(message)
        
        messages.append(message)
        messages.append(tool_response)
        
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()